# Juliaで100本ノック(1-25)

## 準備

In [ ]:
ENV["COLUMNS"]=240  # 描画する表の列数を増やす
ENV["LINES"]=10  # 行の数は制限（問題の指示とは異なるので好みに合わせて修正）

using Pkg

Pkg.add("DataFrames")
Pkg.add("DataFramesMeta")
Pkg.add("LibPQ")
Pkg.add("StatsBase")

using DataFrames
using DataFramesMeta
using LibPQ
using StatsBase
using Statistics

## SQLとの接続

In [ ]:
host = "db"
port = ENV["PG_PORT"]
database = ENV["PG_DATABASE"]
user = ENV["PG_USER"]
password = ENV["PG_PASSWORD"]
dsl = "postgresql://$user:$password@$host:$port/$database"
conn = LibPQ.Connection(dsl)

df_customer = DataFrame(execute(conn, "select * from customer"))
df_category = DataFrame(execute(conn, "select * from category"))
df_product = DataFrame(execute(conn, "select * from product"))
df_receipt = DataFrame(execute(conn, "select * from receipt"))
df_store = DataFrame(execute(conn, "select * from store"))
df_geocode = DataFrame(execute(conn, "select * from geocode"));

## 本編

### 001

In [ ]:
first(df_receipt, 10)

### 002

In [ ]:
# 列名はコロンをつけたSymbol型で表す
first(df_receipt, 10)[:, [:sales_ymd, :customer_id, :product_cd, :amount]]

### 003

In [ ]:
# selectを使うが、デフォルトだとデータをコピーしてしまうのでcopycols=falseを渡す
# （いちいち指定するのは面倒なのでこれより後の問題では気にしていない）
select(first(df_receipt, 10), :sales_ymd => :sales_date, :customer_id, :product_cd, :amount, copycols=false)

### 004

In [ ]:
# クエリーしたいときは他のパッケージを使う。ここではDataFramesMetaを使う。比較演算子をelement-wiseに適用するため.==や.>などドットをつけることに注意
@where(df_receipt[:, [:sales_ymd, :customer_id, :product_cd, :amount]],
    :customer_id.=="CS018205000001")

### 005

In [ ]:
@where(df_receipt[:, [:sales_ymd, :customer_id, :product_cd, :amount]],
    :customer_id.=="CS018205000001",
    :amount.>=1000)

### 006

In [ ]:
# 複雑になってきたのでlinqで書く。orは.|で表す。
@linq df_receipt |>
    select(:sales_ymd, :customer_id, :product_cd, :quantity, :amount) |>
    where(:customer_id.=="CS018205000001", (:amount .>= 1000) .| (:quantity .>= 5))

### 007

In [ ]:
# and条件はカンマで区切って並べるだけ。
@linq df_receipt |>
    select(:sales_ymd, :customer_id, :product_cd, :amount) |>
    where(:customer_id.=="CS018205000001", :amount .>= 1000, :amount .<= 2000)

### 008

In [ ]:
@linq df_receipt |>
    select(:sales_ymd, :customer_id, :product_cd, :amount) |>
    where(:customer_id.=="CS018205000001", :product_cd .!= "P071401019")

### 009

In [ ]:
# 前提が成り立たないのでスキップ

### 010

In [ ]:
# occursinをelement-wiseに使う。列名を指定するとその中身が展開されるのはここまでと同じ。
@linq df_store |>
    where(occursin.(r"^S14", :store_cd)) |>
    first(10)

### 011

In [ ]:
# firstなどdataframe本体の機能もlinqで使える。
@linq df_customer |>
    where(occursin.(r"1$", :customer_id)) |>
    first(10)

### 012

In [ ]:
# 簡単なのでいったんlinqをやめてみる
@where(df_store, occursin.(r"横浜市", :address))

### 013

In [ ]:
@linq df_customer |>
    where(occursin.(r"^[A-F]", :status_cd)) |>
    first(10)

### 014

In [ ]:
@linq df_customer |>
    where(occursin.(r"[1-9]$", :status_cd)) |>
    first(10)

### 015

In [ ]:
# ひとつのwhereの中で処理しないで複数重ねてもよい
@linq df_customer |>
    where(occursin.(r"^[A-F]", :status_cd)) |>    
    where(occursin.(r"[1-9]$", :status_cd)) |>
    first(10)

### 016

In [ ]:
@where(df_store, occursin.(r"\d{3}-\d{3}-\d{4}", :tel_no))

### 017

In [ ]:
@linq df_customer |>
    orderby(:birth_day) |>
    first(10)

### 018

In [ ]:
# DataFrameMetaのorderbyでは降順ソートができないためDataFrameのsortを使う
first(sort(df_customer, order(:birth_day, rev=true)), 10)

### 019

In [ ]:
# とはいえ、DataFrameのsortをlinqの中で使うこともできる。そしてStatsBase.competerankで同率ありの順位を出す。
@linq df_receipt |>
    sort(order(:amount, rev=true)) |>
    transform(rank=competerank(:amount, rev=true)) |>
    select(:customer_id, :amount, :rank) |>
    first(10)

### 020

In [ ]:
# StatsBase.ordinalrankで同率なしの順位を出す。
@linq df_receipt |>
    sort(order(:amount, rev=true)) |>
    transform(rank=ordinalrank(:amount, rev=true)) |>
    select(:customer_id, :amount, :rank) |>
    first(10)

### 021

In [ ]:
# length(df)はできないが、size(df)でもよい。あるいは特定の列に対してならlengthも使える。
nrow(df_receipt)

### 022

In [ ]:
nrow(unique(df_receipt, :customer_id))

### 023

In [ ]:
# combineの使い方さえわかれば簡単
@linq df_receipt |>
    select(:store_cd, :amount, :quantity) |>
    groupby(:store_cd) |>
    combine(:amount => sum, :quantity => sum) |>
    orderby(:store_cd)

### 024

In [ ]:
@linq df_receipt |>
    select(:customer_id, :sales_ymd) |>
    orderby(:customer_id, :sales_ymd) |>
    groupby(:customer_id) |>
    combine(:sales_ymd => last) |>
    first(10)

### 025

In [ ]:
@linq df_receipt |>
    select(:customer_id, :sales_ymd) |>
    orderby(:customer_id, :sales_ymd) |>
    groupby(:customer_id) |>
    combine(:sales_ymd => first) |>
    first(10)